## Assignment : 14

<pre>
1. You can work with preprocessed_data.csv for the assignment. You can get the data from - <a href='https://drive.google.com/drive/u/0/folders/1CJnItndeSSJu7aragQoXWZS9-0apN6pp'>Data folder </a>
2. Load the data in your notebook.
3. After step 2 you have to train 3 types of models as discussed below. 
4. For all the model use <a href='https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics'>'auc'</a> as a metric. check <a  href='https://stackoverflow.com/a/46844409'>this</a> and <a  href='https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/80807'>this</a> for using auc as a metric 
5. You are free to choose any number of layers/hiddden units but you have to use same type of architectures shown below. 
6. You can use any one of the optimizers and choice of Learning rate and momentum.
7. For all the model's use <a href='https://www.youtube.com/watch?v=2U6Jl7oqRkM'>TensorBoard</a> and plot the Metric value and Loss with epoch. While submitting, take a screenshot of plots and include those images in a separate pad and write your observations about them.
8. Make sure that you are using GPU to train the given models.
</pre>

In [1]:
#you can use gdown modules to import dataset for the assignment
#for importing any file from drive to Colab you can write the syntax as !gdown --id file_id
#you can run the below cell to import the required preprocessed data.csv file and glove vector

## <font color='red'> Model-1 </font>
Build and Train deep neural network as shown below

<img src='https://i.imgur.com/w395Yk9.png'>
ref: https://i.imgur.com/w395Yk9.png

- __Input_seq_total_text_data__ --- You have to give Total text data columns. After this use the Embedding layer to get word vectors. Use given predefined glove word vectors, don't train any word vectors. After this use LSTM and get the LSTM output and Flatten that output. 
- __Input_school_state__ --- Give 'school_state' column as input to embedding layer and Train the Keras Embedding layer. 
- __Project_grade_category__  --- Give 'project_grade_category' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_categories__ --- Give 'input_clean_categories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_clean_subcategories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_teacher_prefix' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_remaining_teacher_number_of_previously_posted_projects._resource_summary_contains_numerical_digits._price._quantity__ ---concatenate remaining columns and add a Dense layer after that. 



Below is an example of embedding layer for a categorical columns. In below code all are dummy values, we gave only for referance. 

### 1. Go through this blog, if you have any doubt on using predefined Embedding values in Embedding layer - https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
### 2. Please go through this link https://keras.io/getting-started/functional-api-guide/ and check the 'Multi-input and multi-output models' then you will get to know how to give multiple inputs. 

# <font color='red'> Model-1 </font>

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import SpatialDropout1D, LSTM, BatchNormalization,concatenate,Flatten,Embedding,Dense,Dropout,MaxPooling2D,Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import he_normal,glorot_normal
from tensorflow.keras.regularizers import l1,l2,l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,LearningRateScheduler
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LeakyReLU,Concatenate
from tensorflow.keras.initializers import he_normal,glorot_normal
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.initializers import he_normal
import datetime

C:\Users\Shivanshu\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Shivanshu\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Shivanshu\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
#read the csv file
import pandas as pd
df = pd.read_csv('preprocessed_data.csv')

In [3]:
# perform stratified train test split on the dataset
X = df.drop(['project_is_approved'],axis=1)
y = df['project_is_approved'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)

In [4]:
print(X_train.shape,' ',y_train.shape)
print(X_test.shape,' ',y_test.shape)

(87398, 8)   (87398,)
(21850, 8)   (21850,)


In [5]:
df.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay', 'price'],
      dtype='object')

- Converting y_train & y_test into categorical(One hot encoding)

In [30]:
from tensorflow.keras.utils import to_categorical
y_binary_train = to_categorical(y_train, 2) 
y_binary_test = to_categorical(y_test, 2)

## 1.1 Text Vectorization

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.essay)

vocab_size_essay = len(tokenizer.word_index) + 1
print(vocab_size_essay)

51691


In [8]:
x_train_essay = tokenizer.texts_to_sequences(X_train.essay)
x_test_essay = tokenizer.texts_to_sequences(X_test.essay)

max_essay_length = max(len(elem) for elem in x_train_essay)
print(max_essay_length)

339


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# We pad our input sequences equal to the length of the longest Text sequence
x_train_essay = pad_sequences(x_train_essay, maxlen=max_essay_length)
x_test_essay = pad_sequences(x_test_essay, maxlen=max_essay_length)

In [10]:
import pickle
with open('glove_vectors', 'rb') as f:
  model = pickle.load(f)
  glove_words = set(model.keys())

In [11]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size_essay, 300))
for word, index in tokenizer.word_index.items():
  if word in glove_words:
    embedding_vector = model[word]
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector

## 1.2 Categorical feature Vectorization

In [6]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(X_train.school_state)

x_train_ss = lb.transform(X_train.school_state)
x_test_ss = lb.transform(X_test.school_state)

In [7]:
vocab_size_school_state = len(lb.classes_)
max_length_of_school_state_data = 1

In [8]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(X_train.project_grade_category)

x_train_grade = lb.transform(X_train.project_grade_category)
x_test_grade = lb.transform(X_test.project_grade_category)

In [9]:
vocab_size_project_grade_category = len(lb.classes_)
max_length_of_project_grade_category_data = 1

In [16]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(X_train.teacher_prefix)

x_train_teacher = lb.transform(X_train.teacher_prefix)
x_test_teacher = lb.transform(X_test.teacher_prefix)

vocab_size_teacher_prefix = len(lb.classes_)
max_length_of_teacher_prefix_data = 1

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(X_train.clean_categories)

vocab_size_clean_categories = len(tokenizer.word_index)
x_train_cc = tokenizer.texts_to_sequences(X_train.clean_categories)
x_test_cc = tokenizer.texts_to_sequences(X_test.clean_categories)

max_length_of_cat_data = max(len(elem) for elem in x_train_cc)
print(max_length_of_cat_data)
#padding
x_train_cc = pad_sequences(x_train_cc, maxlen=max_length_of_cat_data)
x_test_cc = pad_sequences(x_test_cc, maxlen=max_length_of_cat_data)

3


In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(X_train.clean_subcategories)

vocab_size_clean_subcategories = len(tokenizer.word_index)
x_train_ccsub = tokenizer.texts_to_sequences(X_train.clean_subcategories)
x_test_ccsub = tokenizer.texts_to_sequences(X_test.clean_subcategories)

max_length_of_cat_ccsub_data = max(len(elem) for elem in x_train_ccsub)
print(max_length_of_cat_ccsub_data)

x_train_ccsub = pad_sequences(x_train_ccsub, maxlen=max_length_of_cat_ccsub_data)
x_test_ccsub = pad_sequences(x_test_ccsub, maxlen=max_length_of_cat_ccsub_data)

3


## 1.3 Numerical feature Vectorization

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train.teacher_number_of_previously_posted_projects.values.reshape(-1, 1))

train_tnoppp = scaler.transform(X_train.teacher_number_of_previously_posted_projects.values.reshape(-1, 1))
test_tnoppp = scaler.transform(X_test.teacher_number_of_previously_posted_projects.values.reshape(-1, 1))

In [13]:
scaler = MinMaxScaler()
scaler.fit(X_train.price.values.reshape(-1, 1))

train_price = scaler.transform(X_train.price.values.reshape(-1, 1))
test_price = scaler.transform(X_test.price.values.reshape(-1, 1))

In [14]:
train_merged_num_features = np.hstack((train_tnoppp, train_price))
test_merged_num_features = np.hstack((test_tnoppp, test_price))

## 1.4 Defining the model

<img src='https://i.imgur.com/w395Yk9.png'>

- Custom roc_auc_score https://stackoverflow.com/a/51734992

In [15]:
def auc1(y_true, y_pred):
    if len(np.unique(y_true[:,1])) == 1:
        return 0.5
    else:
        return roc_auc_score(y_true, y_pred)

def auroc(y_true, y_pred):
    return tf.py_function(auc1, (y_true, y_pred), tf.double)

In [24]:
tf.keras.backend.clear_session()

Input_seq_total_text_data = Input(shape=(max_essay_length,), name="text_input")
embedding_layer = Embedding(vocab_size_essay, 300, weights=[embedding_matrix], input_length=max_essay_length, trainable=False)(Input_seq_total_text_data)
lstm_out = LSTM(128, kernel_initializer='glorot_normal', kernel_regularizer=l2(0.0006),return_sequences=True)(embedding_layer)
flatten_out_text_data = Flatten()(lstm_out)


Input_school_state = Input(shape=(max_length_of_school_state_data,), name="school_state_input")
embedding_layer = Embedding(vocab_size_school_state, 2, input_length=1)(Input_school_state)
flatten_school_state = Flatten()(embedding_layer)


Input_grade_category = Input(shape=(max_length_of_project_grade_category_data,), name="project_grade_category_input")
embedding_layer = Embedding(vocab_size_project_grade_category, 2, input_length=1)(Input_grade_category)
flatten_grade_category = Flatten()(embedding_layer)


Input_teacher_prefix = Input(shape=(max_length_of_teacher_prefix_data,), name="teacher_prefix_input")
embedding_layer = Embedding(vocab_size_teacher_prefix, 2, input_length=1)(Input_teacher_prefix)
flatten_teacher_prefix = Flatten()(embedding_layer)


Input_clean_categories = Input(shape=(max_length_of_cat_data,), name="clean_categories_input")
embedding_layer = Embedding(vocab_size_clean_categories, 2, input_length=max_length_of_cat_data)(Input_clean_categories)
flatten_clean_categories = Flatten()(embedding_layer)


Input_clean_subcategories = Input(shape=(max_length_of_cat_ccsub_data,), name="clean_subcategories_input")
embedding_layer = Embedding(vocab_size_clean_subcategories, 2, input_length=max_length_of_cat_ccsub_data)(Input_clean_subcategories)
flatten_clean_subcategories = Flatten()(embedding_layer)


Input_merged_num_features = Input(shape=(2,), name="merged_num_features_input")
output_merged_num_features = Dense(20, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0003))(Input_merged_num_features)

concat_layer = Concatenate()([flatten_out_text_data, flatten_school_state, flatten_grade_category, flatten_clean_categories, flatten_clean_subcategories, flatten_teacher_prefix, output_merged_num_features])

# We stack a deep densely-connected network on top
x = Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(concat_layer)
x = Dropout(0.15)(x)

x = Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(32, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)
# And finally we add the main softmax regression layer
main_output = Dense(2, activation='softmax', kernel_initializer="glorot_uniform", name='main_output')(x)

checkpoint1 = ModelCheckpoint("model11_final.h5",monitor="val_auroc",mode="max",save_best_only = True,verbose=1)
earlystop1 = EarlyStopping(monitor = 'val_auroc', mode="max",min_delta = 0, patience = 5,verbose = 1)
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=0, write_graph=True,write_grads=True)
callbacks_1 = [tensorboard_callback,earlystop1,checkpoint1]

model = Model(inputs=[Input_seq_total_text_data, Input_school_state, Input_grade_category, Input_clean_categories, Input_clean_subcategories, Input_teacher_prefix, Input_merged_num_features],outputs=main_output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 339)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 339, 300)     15462900    ['text_input[0][0]']             
                                                                                                  
 school_state_input (InputLayer  [(None, 1)]         0           []                               
 )                                                                                                
                                                                                                  
 project_grade_category_input (  [(None, 1)]         0           []                           

## 1.5 Compiling and fititng your model

In [25]:
model.compile(optimizer=Adam(learning_rate=0.0009), loss='categorical_crossentropy', metrics=[auroc])
history = model.fit([x_train_essay, x_train_ss, x_train_grade, x_train_cc, x_train_ccsub, x_train_teacher, train_merged_num_features], y_binary_train,batch_size=900,epochs=40,callbacks=callbacks_1,validation_data=([x_test_essay, x_test_ss, x_test_grade, x_test_cc, x_test_ccsub, x_test_teacher, test_merged_num_features], y_binary_test))

Epoch 1/40
98/98 [==============================] - ETA: 0s - loss: 0.6291 - auroc: 0.6177
Epoch 00001: val_auroc improved from -inf to 0.69812, saving model to model11_final.h5
98/98 [==============================] - 38s 312ms/step - loss: 0.6291 - auroc: 0.6177 - val_loss: 0.5051 - val_auroc: 0.6981
Epoch 2/40
98/98 [==============================] - ETA: 0s - loss: 0.4816 - auroc: 0.7008
Epoch 00002: val_auroc improved from 0.69812 to 0.70580, saving model to model11_final.h5
98/98 [==============================] - 30s 306ms/step - loss: 0.4816 - auroc: 0.7008 - val_loss: 0.4688 - val_auroc: 0.7058
Epoch 3/40
98/98 [==============================] - ETA: 0s - loss: 0.4480 - auroc: 0.7182
Epoch 00003: val_auroc improved from 0.70580 to 0.73377, saving model to model11_final.h5
98/98 [==============================] - 30s 307ms/step - loss: 0.4480 - auroc: 0.7182 - val_loss: 0.4350 - val_auroc: 0.7338
Epoch 4/40
98/98 [==============================] - ETA: 0s - loss: 0.4303 - auroc

In [26]:
score = model.evaluate([x_test_essay, x_test_ss, x_test_grade, x_test_cc, x_test_ccsub, x_test_teacher, test_merged_num_features], y_binary_test, batch_size=700)

print('Test loss:', score[0])
print('Test AUC:', score[1])

32/32 [==============================] - 3s 82ms/step - loss: 0.4618 - auroc: 0.7221
Test loss: 0.46183380484580994
Test AUC: 0.7221055030822754


#### Model 1 </br>
- Test AUC = 0.72


# <font color='red'> Model-2 </font>

Use the same model as above but for 'input_seq_total_text_data' give only some words in the sentance not all the words. Filter the words as below. 

<pre>
1. Fit TF-IDF vectorizer on the Train data <br>
2. Get the idf value for each word we have in the train data. Please go through <a  href='https://stackoverflow.com/questions/23792781/tf-idf-feature-weights-using-sklearn-feature-extraction-text-tfidfvectorizer'>this</a><br>

3. Do some analysis on the Idf values and based on those values choose the low and high threshold value. Because very 
frequent words and very very rare words don't give much information.
Hint - A preferable IDF range is 2-11 for model 2. <br>
4.Remove the low idf value and high idf value words from the train and test data. You can go through each of the
sentence of train and test data and include only those features(words) which are present in the defined IDF range.
5. Perform tokenization on the modified text data same as you have done for previous model.
6. Create embedding matrix for model 2 and then use the rest of the features similar to previous model.
7. Define the model, compile and fit the model.
</pre>

In [23]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(X_train.essay)
data = {'word': vectorizer.get_feature_names() , 'idf_value': vectorizer.idf_}
tfidf_df = pd.DataFrame(data=data)

In [24]:
print("Min tf-idf : ",min(tfidf_df['idf_value']))
print("Max tf-idf : ",max(tfidf_df['idf_value']))

Min tf-idf :  1.0077877719582724
Max tf-idf :  11.685091939370627


In [26]:
filter =   (tfidf_df['idf_value']>=1.007) & (tfidf_df['idf_value'] <=11.68)
tfidf_best = tfidf_df[filter]
print(tfidf_best.shape)

(31444, 2)


In [27]:
Best_tfidf = tfidf_best['word'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(Best_tfidf)

X_train_essay_tfidf  = tokenizer.texts_to_sequences(X_train.essay)
X_test_essay_tfidf   = tokenizer.texts_to_sequences(X_test.essay)

In [28]:
vocab_size_essay = len(tokenizer.word_index) + 1
max_essay_length_tfidf = max(len(elem) for elem in X_train_essay_tfidf)
print(max_essay_length_tfidf)

329


In [29]:
X_train_essay_tfidf = pad_sequences(X_train_essay_tfidf, maxlen=max_essay_length_tfidf)
X_test_essay_tfidf = pad_sequences(X_test_essay_tfidf, maxlen=max_essay_length_tfidf)

In [30]:
import pickle
with open('glove_vectors', 'rb') as f:
  model = pickle.load(f)
  glove_words = set(model.keys())

In [31]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size_essay, 300))
for word, index in tokenizer.word_index.items():
  if word in glove_words:
    embedding_vector = model[word]
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector

In [32]:
tf.keras.backend.clear_session()

Input_seq_total_text_data = Input(shape=(max_essay_length_tfidf,), name="text_input")
embedding_layer = Embedding(vocab_size_essay, 300, weights=[embedding_matrix], input_length=max_essay_length_tfidf, trainable=False)(Input_seq_total_text_data)
embedding_layer = SpatialDropout1D(0.3)(embedding_layer)
lstm_out = LSTM(128, kernel_initializer='he_normal', kernel_regularizer=l2(0.0001),return_sequences=True)(embedding_layer)
flatten_out_text_data = Flatten()(lstm_out)

Input_school_state = Input(shape=(max_length_of_school_state_data,), name="school_state_input")
embedding_layer = Embedding(vocab_size_school_state, 2, input_length=1)(Input_school_state)
flatten_school_state = Flatten()(embedding_layer)


Input_grade_category = Input(shape=(max_length_of_project_grade_category_data,), name="project_grade_category_input")
embedding_layer = Embedding(vocab_size_project_grade_category, 2, input_length=1)(Input_grade_category)
flatten_grade_category = Flatten()(embedding_layer)


Input_teacher_prefix = Input(shape=(max_length_of_teacher_prefix_data,), name="teacher_prefix_input")
embedding_layer = Embedding(vocab_size_teacher_prefix, 2, input_length=1)(Input_teacher_prefix)
flatten_teacher_prefix = Flatten()(embedding_layer)


Input_clean_categories = Input(shape=(max_length_of_cat_data,), name="clean_categories_input")
embedding_layer = Embedding(vocab_size_clean_categories, 2, input_length=max_length_of_cat_data)(Input_clean_categories)
flatten_clean_categories = Flatten()(embedding_layer)


Input_clean_subcategories = Input(shape=(max_length_of_cat_ccsub_data,), name="clean_subcategories_input")
embedding_layer = Embedding(vocab_size_clean_subcategories, 2, input_length=max_length_of_cat_ccsub_data)(Input_clean_subcategories)
flatten_clean_subcategories = Flatten()(embedding_layer)


Input_merged_num_features = Input(shape=(2,), name="merged_num_features_input")
output_merged_num_features = Dense(16, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(Input_merged_num_features)

concat_layer = Concatenate()([flatten_out_text_data, flatten_school_state, flatten_grade_category, flatten_clean_categories, flatten_clean_subcategories, flatten_teacher_prefix, output_merged_num_features])

# stack deep densely-connected network on top
x = Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(concat_layer)
x = Dropout(0.5)(x)

x = Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(32, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)
# the main softmax regression layer
main_output = Dense(2, activation='softmax', kernel_initializer="glorot_uniform", name='main_output')(x)

checkpoint1 = ModelCheckpoint("model21_final.h5",monitor="val_auroc",mode="max",save_best_only = True,verbose=1)
earlystop1 = EarlyStopping(monitor = 'val_auroc', mode="max",min_delta = 0, patience = 5,verbose = 1)
tensorboard1 = TensorBoard(log_dir='logs/train')
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=0, write_graph=True,write_grads=True)
callbacks_1 = [tensorboard_callback,earlystop1,checkpoint1]


model2_1 = Model(inputs=[Input_seq_total_text_data, Input_school_state, Input_grade_category, Input_clean_categories, Input_clean_subcategories, Input_teacher_prefix, Input_merged_num_features],outputs=main_output)
model2_1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 329)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 329, 300)     9433500     ['text_input[0][0]']             
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, 329, 300)    0           ['embedding[0][0]']              
 out1D)                                                                                           
                                                                                                  
 school_state_input (InputLayer  [(None, 1)]         0           []                           

In [33]:
model2_1.compile(optimizer=Adam(learning_rate=0.0006), loss='categorical_crossentropy', metrics=[auroc])
history2_1 = model2_1.fit([X_train_essay_tfidf, x_train_ss, x_train_grade, x_train_cc, x_train_ccsub, x_train_teacher, train_merged_num_features], y_binary_train,batch_size=800,epochs=50,callbacks=callbacks_1,validation_data=([X_test_essay_tfidf, x_test_ss, x_test_grade, x_test_cc, x_test_ccsub, x_test_teacher, test_merged_num_features], y_binary_test))

Epoch 1/50
110/110 [==============================] - ETA: 0s - loss: 0.7163 - auroc: 0.5175
Epoch 00001: val_auroc improved from -inf to 0.57519, saving model to model21_final.h5
110/110 [==============================] - 35s 280ms/step - loss: 0.7163 - auroc: 0.5175 - val_loss: 0.5682 - val_auroc: 0.5752
Epoch 2/50
110/110 [==============================] - ETA: 0s - loss: 0.5801 - auroc: 0.5208
Epoch 00002: val_auroc did not improve from 0.57519
110/110 [==============================] - 30s 273ms/step - loss: 0.5801 - auroc: 0.5208 - val_loss: 0.5403 - val_auroc: 0.5274
Epoch 3/50
110/110 [==============================] - ETA: 0s - loss: 0.5387 - auroc: 0.5277
Epoch 00003: val_auroc did not improve from 0.57519
110/110 [==============================] - 30s 274ms/step - loss: 0.5387 - auroc: 0.5277 - val_loss: 0.5197 - val_auroc: 0.4570
Epoch 4/50
110/110 [==============================] - ETA: 0s - loss: 0.5019 - auroc: 0.5857
Epoch 00004: val_auroc improved from 0.57519 to 0.687

In [34]:
score = model2_1.evaluate([X_test_essay_tfidf, x_test_ss, x_test_grade, x_test_cc, x_test_ccsub, x_test_teacher, test_merged_num_features], y_binary_test, batch_size=800)

print('Test loss:', score[0])
print('Test AUC:', score[1])

28/28 [==============================] - 2s 78ms/step - loss: 0.4028 - auroc: 0.7623
Test loss: 0.4028013348579407
Test AUC: 0.7622679471969604


#### Model 2 </br>
- Test AUC : 0.76

# <font color='red'> Model-3 </font>

<img src='https://i.imgur.com/fkQ8nGo.png'>
ref: https://i.imgur.com/fkQ8nGo.png

In [16]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['school_state'].values)
x_train_state_one_hot = vectorizer.transform(X_train['school_state'].values)
x_test_state_one_hot = vectorizer.transform(X_test['school_state'].values)

In [17]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['clean_categories'].values)
x_train_categories_one_hot = vectorizer.transform(X_train['clean_categories'].values)
x_test_categories_one_hot = vectorizer.transform(X_test['clean_categories'].values)

In [18]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['clean_subcategories'].values)
x_train_subcategories_one_hot = vectorizer.transform(X_train['clean_subcategories'].values)
x_test_subcategories_one_hot = vectorizer.transform(X_test['clean_subcategories'].values)

In [19]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['teacher_prefix'].values)
x_train_teacher_prefix_one_hot = vectorizer.transform(X_train['teacher_prefix'].values)
x_test_teacher_prefix_one_hot = vectorizer.transform(X_test['teacher_prefix'].values)

In [20]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['project_grade_category'].values)
x_train_project_grade_one_hot = vectorizer.transform(X_train['project_grade_category'].values)
x_test_project_grade_one_hot = vectorizer.transform(X_test['project_grade_category'].values)

In [21]:
from scipy.sparse import hstack
train_features_withouttext = hstack((x_train_project_grade_one_hot,x_train_teacher_prefix_one_hot,x_train_categories_one_hot,x_train_subcategories_one_hot,x_train_state_one_hot,train_price,train_tnoppp)).todense()
test_features_withouttext = hstack((x_test_project_grade_one_hot,x_test_teacher_prefix_one_hot,x_test_categories_one_hot,x_test_subcategories_one_hot,x_test_state_one_hot,test_price,test_tnoppp)).todense()

In [22]:
rest_features_train = np.expand_dims(train_features_withouttext,2)
rest_features_test = np.expand_dims(test_features_withouttext,2)
print("Train data ",rest_features_train.shape)
print("Test data s",rest_features_test.shape)

Train data  (87398, 101, 1)
Test data s (21850, 101, 1)


#### Model 3 - batch Size = 64

In [27]:
tf.keras.backend.clear_session()

Input_seq_total_text_data = Input(shape=(max_essay_length,), name="text_input")
embedding_layer = Embedding(vocab_size_essay, 300, weights=[embedding_matrix], input_length=max_essay_length, trainable=False)(Input_seq_total_text_data)
lstm_out = LSTM(20, kernel_initializer='glorot_normal', kernel_regularizer=l1_l2(0.001),return_sequences=True)(embedding_layer)
flatten_out_text_data = Flatten()(lstm_out)

input_without_text  = Input(shape=(101,1), name='without_text')
convo = Conv1D(128 , 3 , activation='relu' ,  kernel_initializer=he_normal(seed=None) , padding='valid')(input_without_text)
convo = Conv1D(128 , 3 , activation='relu' ,  kernel_initializer=he_normal(seed=None) , padding='valid')(convo)
flatten_without_text = Flatten()(convo)

x_concat = concatenate([flatten_out_text_data  , flatten_without_text])

x = Dense(20, activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(x_concat)
x=Dropout(0.1)(x)
x = Dense(20,activation="sigmoid",kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x=Dropout(0.1)(x)
x = Dense(20,activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.001))(x)

output = Dense(2, activation='softmax', name='output')(x)
model_3_1 = Model(inputs=[Input_seq_total_text_data, input_without_text],outputs=[output])

checkpoint1 = ModelCheckpoint("model3_1.h5",monitor="val_auroc",mode="max",save_best_only = True,verbose=1)
earlystop1 = EarlyStopping(monitor = 'val_auroc', mode="max",min_delta = 0, patience = 5,verbose = 1)
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=0, write_graph=True,write_grads=True)
callbacks_1 = [tensorboard_callback,earlystop1,checkpoint1]

model_3_1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 339)]        0           []                               
                                                                                                  
 without_text (InputLayer)      [(None, 101, 1)]     0           []                               
                                                                                                  
 embedding (Embedding)          (None, 339, 300)     15507300    ['text_input[0][0]']             
                                                                                                  
 conv1d (Conv1D)                (None, 99, 128)      512         ['without_text[0][0]']           
                                                                                              

In [28]:
model_3_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[auroc])
history_3_1 = model_3_1.fit([x_train_essay,rest_features_train], y_binary_train, batch_size=64, epochs=35, verbose=1,callbacks=callbacks_1, validation_data=([x_test_essay,rest_features_test], y_binary_test))

Epoch 1/35
1366/1366 [==============================] - ETA: 0s - loss: 0.6020 - auroc: 0.5348
Epoch 00001: val_auroc improved from -inf to 0.56128, saving model to model3_1.h5
1366/1366 [==============================] - 35s 24ms/step - loss: 0.6020 - auroc: 0.5348 - val_loss: 0.4602 - val_auroc: 0.5613
Epoch 2/35
1366/1366 [==============================] - ETA: 0s - loss: 0.4508 - auroc: 0.5659
Epoch 00002: val_auroc did not improve from 0.56128
1366/1366 [==============================] - 30s 22ms/step - loss: 0.4508 - auroc: 0.5659 - val_loss: 0.4468 - val_auroc: 0.5398
Epoch 3/35
1366/1366 [==============================] - ETA: 0s - loss: 0.4369 - auroc: 0.5917
Epoch 00003: val_auroc improved from 0.56128 to 0.66517, saving model to model3_1.h5
1366/1366 [==============================] - 30s 22ms/step - loss: 0.4369 - auroc: 0.5917 - val_loss: 0.4486 - val_auroc: 0.6652
Epoch 4/35
1366/1366 [==============================] - ETA: 0s - loss: 0.4145 - auroc: 0.7046
Epoch 00004: v

In [29]:
score = model_3_1.evaluate([x_test_essay,rest_features_test], y_binary_test, batch_size=64)

print('Test loss:', score[0])
print('Test AUC:', score[1])

342/342 [==============================] - 4s 12ms/step - loss: 0.4186 - auroc: 0.7213
Test loss: 0.4185788631439209
Test AUC: 0.7212847471237183


#### Model 3 - batch size = 900

In [23]:
tf.keras.backend.clear_session()

Input_seq_total_text_data = Input(shape=(max_essay_length,), name="text_input")
embedding_layer = Embedding(vocab_size_essay, 300, weights=[embedding_matrix], input_length=max_essay_length, trainable=False)(Input_seq_total_text_data)
embedding_layer = SpatialDropout1D(0.15)(embedding_layer)
lstm_out = LSTM(128, kernel_initializer='he_normal', kernel_regularizer=l2(0.0007),return_sequences=True)(embedding_layer)
flatten_out_text_data = Flatten()(lstm_out)

input_without_text  = Input(shape=(101,1), name='without_text')
convo = Conv1D(100 , 8 , activation='relu' ,  kernel_initializer=he_normal(seed=None) , padding='valid')(input_without_text)
convo = Conv1D(80 , 8 , activation='relu' ,  kernel_initializer=he_normal(seed=None) , padding='valid')(convo)
flatten_without_text = Flatten()(convo)

x_concat = concatenate([flatten_out_text_data  , flatten_without_text])

x = Dense(128, activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.0005))(x_concat)
x=Dropout(0.15)(x)
x = Dense(64,activation="sigmoid",kernel_initializer="he_normal" ,kernel_regularizer=l2(0.0004))(x)
x = BatchNormalization()(x)
x=Dropout(0.2)(x)
x = Dense(32,activation="relu", kernel_initializer="he_normal" ,kernel_regularizer=l2(0.0001))(x)

output = Dense(2, activation='softmax', name='output')(x)
model_3_2 = Model(inputs=[Input_seq_total_text_data, input_without_text],outputs=[output])

checkpoint1 = ModelCheckpoint("model32.h5",monitor="val_auroc",mode="max",save_best_only = True,verbose=1)
earlystop1 = EarlyStopping(monitor = 'val_auroc', mode="max",min_delta = 0, patience = 5,verbose = 1)
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=0, write_graph=True,write_grads=True)
callbacks_1 = [tensorboard_callback,earlystop1,checkpoint1]

model_3_2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 339)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 339, 300)     15507300    ['text_input[0][0]']             
                                                                                                  
 without_text (InputLayer)      [(None, 101, 1)]     0           []                               
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, 339, 300)    0           ['embedding[0][0]']              
 out1D)                                                                                       

In [24]:
model_3_2.compile(optimizer=Adam(learning_rate=0.0009), loss='categorical_crossentropy', metrics=[auroc])
history_3_2 = model_3_2.fit([x_train_essay,rest_features_train], y_binary_train, batch_size=900, epochs=30, verbose=1,callbacks=callbacks_1, validation_data=([x_test_essay,rest_features_test], y_binary_test))

Epoch 1/30
98/98 [==============================] - ETA: 0s - loss: 1.0722 - auroc: 0.5246
Epoch 00001: val_auroc improved from -inf to 0.52122, saving model to model32.h5
98/98 [==============================] - 43s 349ms/step - loss: 1.0722 - auroc: 0.5246 - val_loss: 0.7761 - val_auroc: 0.5212
Epoch 2/30
98/98 [==============================] - ETA: 0s - loss: 0.6394 - auroc: 0.5974
Epoch 00002: val_auroc improved from 0.52122 to 0.68122, saving model to model32.h5
98/98 [==============================] - 32s 329ms/step - loss: 0.6394 - auroc: 0.5974 - val_loss: 0.5516 - val_auroc: 0.6812
Epoch 3/30
98/98 [==============================] - ETA: 0s - loss: 0.5125 - auroc: 0.6688
Epoch 00003: val_auroc improved from 0.68122 to 0.71076, saving model to model32.h5
98/98 [==============================] - 32s 330ms/step - loss: 0.5125 - auroc: 0.6688 - val_loss: 0.4726 - val_auroc: 0.7108
Epoch 4/30
98/98 [==============================] - ETA: 0s - loss: 0.4684 - auroc: 0.6921
Epoch 000

In [27]:
score = model_3_2.evaluate([x_test_essay,rest_features_test], y_binary_test, batch_size=700)

print('Test loss:', score[0])
print('Test AUC:', score[1])

32/32 [==============================] - 3s 98ms/step - loss: 0.3825 - auroc: 0.7700
Test loss: 0.38254308700561523
Test AUC: 0.7700203061103821


### Model 3 </br>
- Test AUC : 0.77 